## Links and papers

[Dataset on HuggingFace with CATEGORY-level sentiment](https://huggingface.co/datasets/NazmusAshrafi/MAMS_ACSA_SETFITABSA?row=31)

[Dataset on HF - MAMS](https://huggingface.co/datasets/NEUDM/mams)

https://github.com/SCIR-MSA-Team/LC-ACSA

https://github.com/l294265421/ACSA

https://arxiv.org/abs/2206.13969

## More advanced approaches

https://aclanthology.org/2021.emnlp-main.361.pdf

https://paperswithcode.com/paper/sentence-constituent-aware-aspect-category

https://aclanthology.org/2020.coling-main.72.pdf

In [1]:
!pip install datasets==2.16



In [2]:
!pip install fsspec==2023.9.2

In [3]:
import numpy as np
import pandas as pd

from datasets import load_dataset

In [ ]:
!pip list

In [ ]:
#data = load_dataset("NEUDM/mams")

#print(data, data["train"], data["train"][0])

In [ ]:
!python --version
!pip list


In [5]:
data = load_dataset("NazmusAshrafi/MAMS_ACSA_SETFITABSA")

In [6]:
print(data, data["train"], data["train"][0])

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'span', 'label', 'ordinal'],
        num_rows: 7090
    })
}) Dataset({
    features: ['Unnamed: 0', 'text', 'span', 'label', 'ordinal'],
    num_rows: 7090
}) {'Unnamed: 0': 0, 'text': "It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.", 'span': 'food', 'label': 'positive', 'ordinal': 0}


In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
!pip install transformers evaluate accelerate

In [10]:
# going to try to predict aspect category for now

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [12]:
tokenized_ds = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/7090 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-03-13 23:02:52.955914: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 23:02:52.956022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 23:02:53.112013: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
import evaluate

accuracy = evaluate.load("accuracy")

In [15]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
label2id = {"food": 0, "staff": 1, "miscellaneous":2, "place":3, "service":4, "menu":5}

id2label = {v:k for k,v in label2id.items()}

In [17]:
num_labels = len(label2id)

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from datasets import DatasetDict

ds_train_devtest = tokenized_ds['train'].train_test_split(test_size=0.2, seed=42)
ds_devtest = ds_train_devtest['test'].train_test_split(test_size=0.5, seed=42)


ds_splits = DatasetDict({
    'train': ds_train_devtest['train'],
    'valid': ds_devtest['train'],
    'test': ds_devtest['test']
})

print("Before:\n", tokenized_ds)
print("After\n", ds_splits)

In [19]:
training_args = TrainingArguments(
    output_dir="test_aspect_category_prediction",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    label_names=["span"],
    report_to="tensorboard",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_splits["train"],
    eval_dataset=ds_splits["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,    
)

trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`span` in this case) have excessive nesting (inputs type `list` where type `int` is expected).